# Parallel simulations

<div class="topic alert alert-block alert-info">
<b>Topics</b>: Parallel solvers.
</div>

Parallel simulations are run with the `'TetOpSplit'` solver. Please note that the solver is still under active development and some of the functionaliies are not yet available or with limited support. Here is the feature status of solver:

- Fully Supported
    - Non-interactive parallel stochastic spatial reaction-difusion-EField simulation (similar to `'Tetexact'`)
- Not Yet Supported
    - Checkpointing
    - Dynamic load balancing
    - Simulation visualization

For more details about the accuracy and performace of the parallel TetOpSplit solver, please check the following papers:

Hepburn, I., Chen, W., and De Schutter, E. (2016). **Accurate reaction-diffusion operator splitting on tetrahedral meshes for parallel stochastic molecular simulations**. J. Chem. Phys. 145, 054118–22. doi:10.1063/1.4960034.

Chen W and De Schutter E (2017). **Parallel STEPS: Large Scale Stochastic Spatial Reaction-Diffusion Simulation with High Performance Computers**. Front. Neuroinform. 11:13. doi: 10.3389/fninf.2017.00013.

## Converting serial simulations to parallel

We will focus here on models involving tetrahedral meshes. So far, in these cases, we created the simulation object with the `'Tetexact'` serial solver. In order to parallelize these simulations, we need to decide on a way to partition the tetrahedral mesh so that each part is run in a separate process. The first thing that we need to do is thus to declare this partition.

### Partitioning the mesh

#### Linear mesh partition

Mesh partitions are usually created by using one of the available dedicated functions. The simplest one consists in partitioning the mesh along a 3D grid:

```python
import steps.interface

from steps.geom import *

# ...

part = LinearMeshPartition(mesh, xbins, ybins, zbins)
```

In this example, we removed the mesh declaration code in order to focus on the partitioning. The `LinearMeshPartition` function takes the mesh as first parameter and then 3 integers that represent the number of bins along the x, y, and z axis respectively. Assuming the mesh is much longer on the x-axis than on the two others, we could parition the mesh in 10 bins along the x-axis only with `LinearMeshPartition(mesh, 10, 1, 1)`.

This partition function first paritions tetrahedrons according to their position. If patches are present in the model, their triangles will automatically be partitioned according to the tetrahedron partitions.

#### Complex mesh partition

The grid-based partitioning approach may not be suitable for complex geometries, such as a dendritic tree, in this case, we suggest using third party partitioning tools such as [Metis](http://glaros.dtc.umn.edu/gkhome/metis/metis/overview).
To partition a STEPS TetMesh using Metis, we first need to convert the tetrahedron connectivity information in the mesh to Metis format:

```python
mesh.ConvertToMetis('/path/to/file/mymesh.metis')
```

We then call Metis from a bash terminal:

```bash
mpmetis -ncommon=3 -minconn -niter=1000 mymesh.metis 10
```

Metis will partition the mesh into 10 segments and store the partition information to "mymesh.metis.epart.10". Details of Metis parameters can be found in the [Metis user manual](http://glaros.dtc.umn.edu/gkhome/fetch/sw/metis/manual.pdf). After the partitioning, you can read the partition information from the file with:

```python
part = MetisPartition('/path/to/file/mymesh.metis.epart.10')
```

#### Manual mesh partition

The `LinearMeshPartition` and `MetisPartition` functions both return a `MeshPartition` object. It is possible to directly construct a `MeshPartition` by explicitely specifying which tetrahedrons and triangles should be attributed to which partition:

```python

def getTetPartition(tet):
    # ...
    
def getTriPartition(tri):
    # ...

tet_hosts = [getTetPartition(tet) for tet in mesh.tets]

tri_hosts = {}
for patch in mesh.ALL(TetPatch):
    for tri in patch.tris:
        tri_hosts[tri.idx] = getTriPartition(tri)

part = MeshPartition(mesh, tet_hosts, tri_hosts)
```

In this example, we assume that we have two functions : `getTetPartition` that takes a tetrahedron as a parameter and returns an integer partition index ; and `getTriPartition` that takes a triangle as a parameter and returns an integer partition index. We then need to create the `tet_hosts` list that will associate each tetrahedron to an integer partition index. All tetrahedrons need to be partitioned so the list is as long as the number of tetrahedrons in the mesh. Triangles only need to be partition if they are part of a `TetPatch`. We thus loop over all `TetPatch` objects in the mesh and build the `tri_hosts` dictionary that associates a triangle index to an integer partition index.

Finally, we create the partition object by simply passing the mesh along with `tet_hosts` and `tri_hosts` to the `MeshPartition` constructor. More details are available in the [documentation](API_geom.rst#steps.API_2.geom.MeshPartition).